# Capstone project
## Segmenting and Clustering Neighborhoods in Toronto

This notebook will be used for the capstone project

Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import bs4 as bs
import requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

!pip install geocoder
import geocoder

!pip install folium==0.5
import folium # map rendering library

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

     |████████████████████████████████| 98 kB 40 kB/s eta 0:00:012
     |████████████████████████████████| 79 kB 8.6 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=1db0ad958a2f39cce1fa2b57709d7a05239e9102f7d446457ce8dc2dcbecbac5
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium


In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [3]:
url="http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Get page content

In [4]:
page = requests.get(url)

Use BeautifulSoup parser

In [5]:
soup = bs.BeautifulSoup(page.text, 'html.parser')

Find table with the class 'wikitable sortable' using find_all method

In [6]:
table = soup.find_all('table',attrs={'class':"wikitable sortable"})
table

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighbourhood
 </th></tr>
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>
 <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>
 <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>
 <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>
 <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue, Humber Valley Village
 </td></tr>
 <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>
 <tr>
 <td>M2B

Find all rows in the table and append each rows into a list

Finally pass the list to the dataframe

In [7]:
table_rows = table[0].find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
neighborhood=pd.DataFrame(l, columns=["Postalcode", "Borough", "Neighborhood"])
neighborhood.head()

,Postalcode,Borough,Neighborhood
0,None,None,None
1,M1A\n,Not assigned\n,Not assigned\n
2,M2A\n,Not assigned\n,Not assigned\n
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n


Remove the first blank row

In [8]:
neighborhood.drop(labels=0,axis=0,inplace=True)
neighborhood.head()

,Postalcode,Borough,Neighborhood
1,M1A\n,Not assigned\n,Not assigned\n
2,M2A\n,Not assigned\n,Not assigned\n
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n
5,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


 Trim \n from ends of each value across all series in dataframe

In [9]:
def trim_all_columns(df):
    """
    Trim \n from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip('\n') if isinstance(x, str) else x
    return df.applymap(trim_strings)

neighhborhood_trimmed = trim_all_columns(neighborhood)
neighhborhood_trimmed

,Postalcode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
176,M5Z,Not assigned,Not assigned
177,M6Z,Not assigned,Not assigned
178,M7Z,Not assigned,Not assigned
179,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Remove all rows with 'Not assigned' in the Borough column

Reset the index

In [10]:
neighhborhood_trimmed=neighhborhood_trimmed.drop(neighhborhood_trimmed[neighhborhood_trimmed.Borough=="Not assigned"].index)
neighhborhood_trimmed.reset_index(drop=True,inplace=True)
neighhborhood_trimmed

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [11]:
neighhborhood_trimmed.shape

(103, 3)

Use Geocoder to find out latitudes and longitudes of all Postalcodes in Toronto

Save the result in a list

In [12]:
# initialize your variable to None
latlng = None
latitude=[]
longitude=[]

for code in neighhborhood_trimmed['Postalcode']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    #print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

Convert the list to a dataframe

In [13]:
neighhborhood_laglng=pd.DataFrame([latitude,longitude],['Latitude','Longitude']).T
neighhborhood_laglng

,Latitude,Longitude
0,43.75245,-79.32991
1,43.73057,-79.31306
2,43.65512,-79.36264
3,43.72327,-79.45042
4,43.66253,-79.39188
...,...,...
98,43.65319,-79.51113
99,43.66659,-79.38133
100,43.64869,-79.38544
101,43.63278,-79.48945


Concatentate the original dataframe with the latitude/longitude dataframe

In [14]:
neighhborhood_final = pd.concat([neighhborhood_trimmed,neighhborhood_laglng], axis=1)
neighhborhood_final.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


Extract rows with the word 'Toronto' in the column Borough

In [15]:
toronto_data=neighhborhood_final[neighhborhood_final.Borough.str.contains("Toronto",case=False)]
toronto_data.reset_index(drop=True,inplace=True)

In [16]:
toronto_data

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
3,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
4,M4E,East Toronto,The Beaches,43.67709,-79.29547
5,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306
6,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493
7,M6G,Downtown Toronto,Christie,43.66869,-79.42071
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.64970,-79.38258
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.66505,-79.43891


#### Use geopy library to get the latitude and longitude values of Toronto city.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>toronto_explorer</em>, as shown below.

In [17]:
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [18]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [19]:
#!pip install folium==0.5
#import folium # map rendering library

let's visualize Toronto and the neighborhoods in it.

In [20]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [21]:
CLIENT_ID = '05KHEHGAUVAEEWVVCG134NG3XC5J0AON1IKXKLE4G05HOMXO' # your Foursquare ID
CLIENT_SECRET = 'R0NBCIROLWDXNOCZDDD2BNF1OHRV0G3YDRCIVO2ZFCDSFCSZ' # your Foursquare Secret
ACCESS_TOKEN = "ERM3ZZZVRZEV2TMGSTTKG2U4TJJVRYRD5AKFRNVCO0R0H4MR" # your Foursquare access token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('ACCESS_TOKEN:' + ACCESS_TOKEN)

Your credentails:
CLIENT_ID: 05KHEHGAUVAEEWVVCG134NG3XC5J0AON1IKXKLE4G05HOMXO
CLIENT_SECRET:R0NBCIROLWDXNOCZDDD2BNF1OHRV0G3YDRCIVO2ZFCDSFCSZ
ACCESS_TOKEN:ERM3ZZZVRZEV2TMGSTTKG2U4TJJVRYRD5AKFRNVCO0R0H4MR


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [22]:
toronto_data.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

Get the neighborhood's latitude and longitude values.


In [23]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.65512000000007, -79.36263999999994.


#### Now, let's get the top 100 venues that are in Regent Park, Harbourfront within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [24]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&oauth_token={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET,
    ACCESS_TOKEN,
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=05KHEHGAUVAEEWVVCG134NG3XC5J0AON1IKXKLE4G05HOMXO&client_secret=R0NBCIROLWDXNOCZDDD2BNF1OHRV0G3YDRCIVO2ZFCDSFCSZ&oauth_token=ERM3ZZZVRZEV2TMGSTTKG2U4TJJVRYRD5AKFRNVCO0R0H4MR&v=20180605&ll=43.65512000000007,-79.36263999999994&radius=500&limit=100'

Send the GET request and examine the resutls

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ff1d4d3570d39545ee540c0'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 40,
  'suggestedBounds': {'ne': {'lat': 43.65962000450007,
    'lng': -79.3564319112327},
   'sw': {'lat': 43.650619995500065, 'lng': -79.36884808876718}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723

Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Berkeley Church,Event Space,43.655123,-79.365873


And how many venues were returned by Foursquare?


In [28]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

40 venues were returned by Foursquare.


## Explore Neighborhoods in Manhattan


#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&oauth_token={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            ACCESS_TOKEN,
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _toronto_venues_.

In [30]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


#### Let's check the size of the resulting dataframe

In [31]:
print(toronto_venues.shape)
toronto_venues.head()

(2258, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65512,-79.36264,Berkeley Church,43.655123,-79.365873,Event Space


Let's check how many venues were returned for each neighborhood

In [32]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",98,98,98,98,98,98
Central Bay Street,100,100,100,100,100,100
Christie,17,17,17,17,17,17
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,45,45,45,45,45,45


#### Let's find out how many unique categories can be curated from all the returned venues

In [33]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


##  Analyze Each Neighborhood

In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [35]:
toronto_onehot.shape

(2258, 272)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [36]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.010000,0.000000,0.000000,0.00,0.00,0.010000,0.010000,0.00,0.00,...,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.010000,0.000000,0.010000,0.00,0.00,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.020000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.00,0.00,0.020000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.010204,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.010204,0.00,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
6,Church and Wellesley,0.020000,0.000000,0.000000,0.01,0.01,0.010000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.00,0.010000,0.010000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.00,0.00,0.030000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.00,0.00,0.022222,0.000000,0.00,0.00,...,0.044444,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.00,0.00,0.066667,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


#### Let's confirm the new size

In [37]:
toronto_grouped.shape

(39, 272)

#### Let's print each neighborhood along with the top 5 most common venues

In [38]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.11
1              Bakery  0.04
2        Cocktail Bar  0.04
3  Seafood Restaurant  0.03
4      Breakfast Spot  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0             Bar  0.07
1      Restaurant  0.05
2            Café  0.05
3     Coffee Shop  0.05
4  Sandwich Place  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.05
2        Hotel  0.04
3          Gym  0.04
4   Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Italian Restaurant  0.06
1         Coffee Shop  0.06
2                 Bar  0.05
3                 Pub  0.05
4           Nightclub  0.05


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.11
1       

#### Let's put that into a _pandas_ dataframe

First, let's write a function to sort the venues in descending order.

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Breakfast Spot,Seafood Restaurant,Restaurant,Bistro,Beer Bar,Cheese Shop,Pharmacy
1,"Brockton, Parkdale Village, Exhibition Place",Bar,Restaurant,Sandwich Place,Café,Coffee Shop,Gift Shop,Japanese Restaurant,Supermarket,Italian Restaurant,Burrito Place
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Café,Hotel,Gym,Restaurant,Steakhouse,Sushi Restaurant,Thai Restaurant,Event Space,Seafood Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Nightclub,Bar,Pub,Café,French Restaurant,Gym / Fitness Center,Sandwich Place,Park
4,Central Bay Street,Coffee Shop,Clothing Store,Sushi Restaurant,Cosmetics Shop,Chinese Restaurant,Café,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Fast Food Restaurant


## Cluster Neighborhoods

Run _k_-means to cluster the neighborhood into 5 clusters.

In [41]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,0,Coffee Shop,Thai Restaurant,Food Truck,Pub,Breakfast Spot,Furniture / Home Store,Grocery Store,Sandwich Place,Restaurant,Rental Car Location
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,0,Coffee Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Mediterranean Restaurant,Portuguese Restaurant,Café,Restaurant,Bar,Sporting Goods Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,0,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Middle Eastern Restaurant,Burger Joint,Department Store
3,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,0,Restaurant,Hotel,Italian Restaurant,Café,Coffee Shop,Gastropub,Diner,Cosmetics Shop,Seafood Restaurant,Movie Theater
4,M4E,East Toronto,The Beaches,43.67709,-79.29547,0,Women's Store,Asian Restaurant,Home Service,Park,College Gym,Pizza Place,Coffee Shop,Pub,Rental Car Location,Health Food Store


Finally, let's visualize the resulting clusters

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

#### Cluster 1


In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Thai Restaurant,Food Truck,Pub,Breakfast Spot,Furniture / Home Store,Grocery Store,Sandwich Place,Restaurant,Rental Car Location
1,Downtown Toronto,0,Coffee Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Mediterranean Restaurant,Portuguese Restaurant,Café,Restaurant,Bar,Sporting Goods Shop
2,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Middle Eastern Restaurant,Burger Joint,Department Store
3,Downtown Toronto,0,Restaurant,Hotel,Italian Restaurant,Café,Coffee Shop,Gastropub,Diner,Cosmetics Shop,Seafood Restaurant,Movie Theater
4,East Toronto,0,Women's Store,Asian Restaurant,Home Service,Park,College Gym,Pizza Place,Coffee Shop,Pub,Rental Car Location,Health Food Store
5,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Breakfast Spot,Seafood Restaurant,Restaurant,Bistro,Beer Bar,Cheese Shop,Pharmacy
6,Downtown Toronto,0,Coffee Shop,Clothing Store,Sushi Restaurant,Cosmetics Shop,Chinese Restaurant,Café,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Fast Food Restaurant
7,Downtown Toronto,0,Café,Grocery Store,Playground,Flower Shop,Coffee Shop,Candy Store,Baby Store,Japanese Restaurant,Pool,Italian Restaurant
8,Downtown Toronto,0,Café,Coffee Shop,Gym,Restaurant,Hotel,Japanese Restaurant,Steakhouse,Salad Place,Asian Restaurant,American Restaurant
9,West Toronto,0,Park,Bakery,Pharmacy,Café,Brazilian Restaurant,Smoke Shop,Mattress Store,Middle Eastern Restaurant,Pool,Bar


#### Cluster 2

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,1,Park,Gym Pool,Women's Store,Escape Room,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School


#### Cluster 3

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,2,Park,Gym,Tennis Court,Playground,Trail,Beer Store,Event Service,Ethiopian Restaurant,Event Space,Falafel Restaurant
33,Downtown Toronto,2,Park,Gym / Fitness Center,Playground,Women's Store,Elementary School,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


#### Cluster 4

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,West Toronto,3,Park,Residential Building (Apartment / Condo),Sandwich Place,Convenience Store,Intersection,Event Service,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant


#### Cluster 5

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Home Service,IT Services,Health & Beauty Service,Escape Room,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School
